# 小提琴图绘制

In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

# 文件路径
root_path = 'your path/Toxic.csv'

# 从 CSV 文件读取数据
data = pd.read_csv(root_path)

# 设置不同组的颜色
r_colors = ['#cecece', '#dd6d7c', '#9ce29c', '#bcdce3', '#e3debc', '#e4eaf2', '#dd6d7c', '#9ce29c', '#bcdce3', '#e3debc', '#e4eaf2']

# 创建图形
plt.figure(figsize=(12, 6))

# 选择数据（这里不做列的筛选，全部使用）
data_subset = data.iloc[:, :]

# 绘制小提琴图并设置颜色
sns.violinplot(data=data_subset, palette=r_colors)

# 在图上添加统计数据
for i, column in enumerate(data_subset.columns):
    mean = data_subset[column].mean()
    plt.text(i, mean, f"Mean: {mean:.2f}", ha='center', va='top', fontweight='bold', fontsize=12)

# 设置横轴标签为两行格式
# 示例：将列名转换为两行格式
new_labels = [f"{label.split('(')[0]}\n({label.split('(')[1]}" if '(' in label else label for label in data.columns]

# 更新刻度标签
plt.xticks(ticks=range(len(new_labels)), labels=new_labels, fontsize=16, rotation=30, ha='right')
# plt.gca().get_xaxis().set_visible(False)
# 设置纵轴字体大小
plt.yticks(fontsize=20)

# 设置y轴标签
plt.ylabel(r'$P_{Toxic}$', fontsize=20)

# 设置标题
plt.title(f'{Path(root_path).stem} Probability Value Distribution', fontsize=20)

# 设置y轴范围
plt.ylim(0, 1)

# 保存图形为 SVG 格式
file_name = os.path.splitext(os.path.basename(root_path))[0]
save_path = f'your path/{file_name}.svg'
plt.savefig(save_path, bbox_inches='tight', format='svg', facecolor='white', dpi=400)

# 显示图形
plt.show()

# 帕累托前沿绘制

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 文件路径
files = ['your path/CVAE.csv',
         'your path/PepCVAE.csv',
        'your path/HydrAMP.csv', 
        # 'your path/ProGen2.csv',
         'your path/PepLoRA.csv',
         'your path/PepDPO.csv', 
         'your path/PepPFDPO.csv',
         ]

# 颜色和图例标记
colors = [
    "#d486bd",  # 蓝色
    '#ff7f0e',  # 橙色
    '#2ca02c',  # 绿色
    # "#f11404",  # 红色
    "#7a00ec",  # 紫色
    '#17becf',  # 棕色
    "#fcf808",  # 黄色
]

# 每种方法对应的点形状
markers = ['o', 's', '^', 'P', 'D', 'X', '*']

# 两两组合定义
combinations = [
    ('AMP', 'MIC', True, True),    # 最大化 AMP 和 MIC
    ('AMP', 'Toxic', True, False), # 最大化 AMP，最小化 Toxic
    ('MIC', 'Toxic', True, False)  # 最大化 MIC，最小化 Toxic
]

# 创建画布
fig, axes = plt.subplots(1, len(combinations), figsize=(18, 6))

# 遍历文件和组合
for file_id, file in enumerate(files):
    data = pd.read_csv(file)

    # 提取仅包含文件名（去掉路径和后缀）
    short_file_name = os.path.splitext(os.path.basename(file))[0].replace('_pred', '')

    for i, (x_label, y_label, maximize_x, maximize_y) in enumerate(combinations):
        x = data[x_label].values
        y = data[y_label].values

        # 主图区域
        main_ax = axes[i]

        # 绘制散点图，使用不同的形状和颜色
        main_ax.scatter(
            x, y, alpha=0.5, label=f'{short_file_name}', 
            color=colors[file_id], marker=markers[file_id], s=50  # 添加点形状和大小
        )

        # 设置轴标签、标题和图例
        main_ax.set_xlabel(x_label, fontsize=18)  # 调大 x 轴标签字体大小
        main_ax.set_ylabel(y_label, fontsize=18)  # 调大 y 轴标签字体大小
        main_ax.legend(loc='upper left', fontsize=12)  # 调整图例字体大小

        # 设置刻度字体大小
        main_ax.tick_params(axis='both', which='major', labelsize=18)  # 刻度字体大小

        # 自动调整坐标轴范围 (去掉多余的空白)
        x_min, x_max = np.min(x), np.max(x)
        y_min, y_max = np.min(y), np.max(y)
        main_ax.set_xlim(x_min - 0.03 * (x_max - x_min), x_max + 0.03 * (x_max - x_min))
        main_ax.set_ylim(y_min - 0.03 * (y_max - y_min), y_max + 0.03 * (y_max - y_min))

        # 密度分布图
        # 上侧横轴的密度分布
        top_ax = main_ax.inset_axes([0, 1.0, 1, 0.2], sharex=main_ax)
        sns.kdeplot(x, ax=top_ax, color=colors[file_id], fill=True, alpha=0.3)
        top_ax.set_xlim(main_ax.get_xlim())  # 确保范围一致
        top_ax.axis('off')  # 隐藏坐标轴

        # 右侧纵轴的密度分布
        right_ax = main_ax.inset_axes([1.0, 0, 0.2, 1], sharey=main_ax)
        sns.kdeplot(y, ax=right_ax, color=colors[file_id], fill=True, alpha=0.3, vertical=True)
        right_ax.set_ylim(main_ax.get_ylim())  # 确保范围一致
        right_ax.axis('off')  # 隐藏坐标轴

# 调整布局并显示
plt.tight_layout()
plt.savefig('your path/paretomin50.svg', bbox_inches='tight', format='svg', facecolor='white', dpi=400)  # 设置dpi为400
plt.show()

# 氨基酸频率计算

In [ ]:
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

# 读取两个 CSV 文件
file1 = "your path/Original.csv"
file2 = "your path/generate.csv"
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# 确保 Sequence 列存在
if 'Sequence' not in df1.columns or 'Sequence' not in df2.columns:
    raise ValueError("One or both files are missing the 'Sequence' column.")

# 统计每个文件中氨基酸的频率
def calculate_aa_frequency(sequence_column):
    # 拼接所有序列成一个字符串
    all_sequences = ''.join(sequence_column)
    # 使用 Counter 统计每个氨基酸的出现次数
    aa_counts = Counter(all_sequences)
    # 计算总氨基酸数量
    total_aa = sum(aa_counts.values())
    # 计算每种氨基酸的频率
    aa_frequency = {aa: count / total_aa for aa, count in aa_counts.items()}
    return aa_frequency

# 计算两个文件的氨基酸频率
freq1 = calculate_aa_frequency(df1['Sequence'])
freq2 = calculate_aa_frequency(df2['Sequence'])

# 转换为 DataFrame 以便对比
freq_df = pd.DataFrame([freq1, freq2], index=['Original', 'FD']).T.reset_index()
freq_df.columns = ['Amino Acid', 'Original', 'FD']

# 绘制条形图
plt.figure(figsize=(12, 6))
freq_df_melted = freq_df.melt(id_vars='Amino Acid', 
                              var_name='File', 
                              value_name='Frequency')

sns.barplot(data=freq_df_melted, x='Amino Acid', y='Frequency', hue='File', palette='Set2')

# 图形美化
plt.title('Amino Acid Frequency Distribution', fontsize=18)
plt.xlabel('Amino Acid', fontsize=18)
plt.ylabel('Frequency', fontsize=18)
plt.legend(title=None, fontsize=14)
plt.grid(alpha=0.3)

# 调整刻度字体大小
plt.tick_params(axis='x', labelsize=18)  # 调整 X 轴刻度字体大小
plt.tick_params(axis='y', labelsize=18)  # 调整 Y 轴刻度字体大小

# 显示图
plt.tight_layout()
# plt.savefig('your path/frequence.svg', bbox_inches='tight', format='svg', facecolor='white', dpi=400)  # 设置dpi为300
plt.show()
